## Σε αυτό το σημείο της εργασίες πρέπει να συγκρίνουμε τα 3 pipelines που χρησιμοποιήσαμε στο ερώτημα 1Β μεταξύ τους

### Βήματα 
- Διαβάζουμε τα 3 ανακατσκευασμένα κείμενα
- Κάνουμε σύγκριση

In [23]:
original_text=""
pipeline1=""
pipeline2=""
pipeline3=""

In [24]:
try:
    with open('../text_files/text1.txt', 'r', encoding='utf-8') as file:
        original_text = file.read()
    print("File content successfully loaded into original_text.")
except FileNotFoundError:
    print("Error: The file 'text1.txt' was not found. Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into original_text.


In [25]:
try:
    with open("../text_files/first_pipeline_text1.txt", 'r', encoding='utf-8') as file:
        pipeline1 = file.read()
    print("File content successfully loaded into pipeline1.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline1.


In [26]:
try:
    with open("../text_files/second_pipeline_text1.txt", 'r', encoding='utf-8') as file:
        pipeline2 = file.read()
    print("File content successfully loaded into pipeline2.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline2.


In [27]:
try:
    with open("../text_files/third_pipeline_text1.txt", 'r', encoding='utf-8') as file:
        pipeline3 = file.read()
    print("File content successfully loaded into pipeline3.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline3.


## Αρχικά θα συγκρίνουμε το πόσο διαφέρει η κάθε ανακατασκευή από το πρωτότυπο κείμενο με μέθοδο cosine similarity

In [28]:
from collections import Counter
import numpy as np
def cosine_similarity_logic(original_text,pipeline):
    sentences = [original_text, pipeline]
    tokenized = [s.lower().split() for s in sentences]
    vocab = sorted(set(word for doc in tokenized for word in doc))

    tf_matrix = []
    for doc in tokenized:
        word_counts = Counter(doc)
        tf_vector = [word_counts[word] / len(doc) for word in vocab]
        tf_matrix.append(tf_vector)
        
        
    N = len(tokenized) 
    df_vector = [sum(1 for doc in tokenized if word in doc) for word in vocab]
    idf_vector = [np.log((N + 1) / (df + 1)) + 1 for df in df_vector]
    tfidf_matrix = np.array(tf_matrix) * np.array(idf_vector)
    
    
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    print(f"Cosine Similarity: {similarity}")
    
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [29]:
cosine_similarity_logic(original_text,pipeline1)
cosine_similarity_logic(original_text,pipeline2)
cosine_similarity_logic(original_text,pipeline3)



Cosine Similarity: 0.909638830217755
Cosine Similarity: 0.89091260117103
Cosine Similarity: 0.6972463548228686


## Ground truth κείμενο από το gemini

In [30]:
gemini="""Happy Dragon Boat Festival! I hope you're celebrating safely and enjoying a wonderful time.

Thank you for relaying our message to the doctor regarding his contract review. I actually received the approved message from the professor a couple of days ago. I'm very grateful for the professor's full support with our Springer proceedings publication."""

In [31]:
cosine_similarity_logic(gemini,pipeline1)
cosine_similarity_logic(gemini,pipeline2)
cosine_similarity_logic(gemini,pipeline3)

Cosine Similarity: 0.4912117852064911
Cosine Similarity: 0.5029258632108523
Cosine Similarity: 0.41372297221366444


## Ground truth κείμενο από το deepseek

In [32]:
deepseek="""Today is our Dragon Boat Festival, a cherished celebration in Chinese culture. I wish you safety, joy, and prosperity during this time—may you enjoy the festivities as deeply as I hope for you.

Thank you for sharing the message regarding the doctor’s contract review. I’ve already received the approved notice from the professor a few days ago and sincerely appreciate their unwavering support for our Springer proceedings publication."""

In [33]:
cosine_similarity_logic(deepseek,pipeline1)
cosine_similarity_logic(deepseek,pipeline2)
cosine_similarity_logic(deepseek,pipeline3)

Cosine Similarity: 0.4240647916966377
Cosine Similarity: 0.4173758627748884
Cosine Similarity: 0.3736299567629978


## Ground truth κείμενο από το chat gpt

In [34]:
chat_gpt="""Today is our Dragon Boat Festival, a special celebration in our Chinese culture. It is a time to honor and wish for safety and prosperity in our lives. I hope you also enjoy this festival, as I send you my warmest wishes.

Thank you for your message and for conveying our words to the doctor regarding his upcoming contract review. This is important for all of us.

I received this message to confirm the approval. Actually, the professor shared this with me a couple of days ago. I truly appreciate the professor’s full support for our Springer proceedings publication."""

In [35]:
cosine_similarity_logic(chat_gpt,pipeline1)
cosine_similarity_logic(chat_gpt,pipeline2)
cosine_similarity_logic(chat_gpt,pipeline3)

Cosine Similarity: 0.6111970629422422
Cosine Similarity: 0.6211219678610983
Cosine Similarity: 0.49879739978083254
